## Data output manipulation

In [148]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import scipy.ndimage
import matplotlib.pyplot as plt
import glob
import re
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from sklearn.decomposition import PCA
from time import time
import math
from sklearn import cross_validation

In [149]:
# constants

data = '/kaggle/dev/data-science-bowl-2017-data/'
stage1 = '/kaggle/dev/data-science-bowl-2017-data/stage1/'
labels = '/kaggle/dev/data-science-bowl-2017-data/stage1_labels.csv'
stage1_processed = '/kaggle/dev/data-science-bowl-2017-data/stage1_processed/'
stage1_features_resnet = '/kaggle/dev/data-science-bowl-2017-data/stage1_features_mx/'
stage1_submission = '/kaggle/dev/data-science-bowl-2017-data/stage1_sample_submission.csv'
naive_submission = '/kaggle/dev/jovan/data-science-bowl-2017/data-science-bowl-2017/submissions/naive_submission.csv'
stage1_processed_pca = '/kaggle/dev/data-science-bowl-2017-data/stage1_processed_pca/'
stage1_features_inception = '/kaggle/dev/data-science-bowl-2017-data/CIFAR-10/cache/'
submissions = '/kaggle/dev/data-science-bowl-2017-data/submissions/'

cifar10_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
cifar_data = "/kaggle/dev/data-science-bowl-2017-data/CIFAR-10/"

In [150]:
ids = list()
for s in glob.glob(stage1_features_inception + "*"):
    id = os.path.basename(s)
    id = re.match(r'inception_cifar10_([a-f0-9].*).pkl' , id).group(1)
    ids.append(id)
ids = pd.DataFrame(ids,  columns=["id"])

df = pd.read_csv(labels)
df = pd.merge(df, ids, how='inner', on=['id'])

x = np.array([np.mean(np.load(stage1_features_inception + "inception_cifar10_" + s + ".pkl"), axis=0) for s in df['id'].tolist()])
y = df['cancer'].as_matrix()
trn_x, val_x, trn_y, val_y = cross_validation.train_test_split(x, y, random_state=42, stratify=y,
                                                                test_size=0.20)

In [151]:
num_classes = 2

transfer_values_test = val_x
transfer_values_train = trn_x
cls_test = val_y
cls_train = trn_y
labels_test = (np.arange(num_classes) == val_y[:, None])+0
labels_train = (np.arange(num_classes) == trn_y[:, None])+0


print("transfer_values_test : " + str(transfer_values_test.shape))
print("transfer_values_train : " + str(transfer_values_train.shape))
print("labels_test : " + str(labels_test.shape))
print("labels_train : " + str(labels_train.shape))
print("cls_test : " + str(cls_test.shape))
print("cls_train : " + str(cls_train.shape))

transfer_values_test : (280, 2048)
transfer_values_train : (1117, 2048)
labels_test : (280, 2)
labels_train : (1117, 2)
cls_test : (280,)
cls_train : (1117,)


In [152]:
labels_test.shape

(280, 2)

In [154]:
print(val_y.shape)
print(np.arange(2))
print(val_y)
print([0,1] == val_y[:,None])

(280,)
[0 1]
[0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0
 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1
 0 0 1 0 0 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0
 1 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 1 0 0 0 0 1 0 1 1 0 0 0 0 1 1 0 0 1 0 1]
[[ True False]
 [False  True]
 [ True False]
 [ True False]
 [ True False]
 [ True False]
 [ True False]
 [ True False]
 [False  True]
 [ True False]
 [ True False]
 [False  True]
 [ True False]
 [False  True]
 [ True False]
 [False  True]
 [False  True]
 [ True False]
 [ True False]
 [False  True]
 [ True False]
 [ True False]
 [ True False]
 [ True False]
 [False  True]
 [ True False]
 [False  True]
 [ True False

In [180]:
x = val_y[1]
print(x)
print(type(x))
print(x.shape)

1
<class 'numpy.int64'>
()
